In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


%pip install --upgrade scikit-learn
%pip install --upgrade imbalanced-learn

from imblearn.over_sampling import RandomOverSampler


ModuleNotFoundError: No module named 'sklearn.utils'

In [ ]:
train_data = pd.read_csv('data/fraudTrain.csv')
test_data = pd.read_csv('data/fraudTest.csv')

train_data['is_train'] = 1
test_data['is_train'] = 0

In [ ]:
split = train_data.size
data = pd.concat([train_data, test_data])

In [ ]:
data.tail()
test_data.head()

In [ ]:
def encode_labels(df, column_name):
    le = preprocessing.LabelEncoder()
    data[column_name] = le.fit_transform(df[column_name])
    return data

def encode_onehot(df, column_name):
    return pd.get_dummies(df, columns=[column_name])

def minmax_scale(df, column_name):
    scaler = preprocessing.MinMacScaler()
    data = scaler.fit_transform(df[column_name])
    return scaler, data

In [ ]:
def convert_epoch(df, column_name):
    for name, value in df[column_name].iteritems():
        value = time.strptime(value)
    return df[column_name]
    
def convert_utc(df, column_name):
    for name, value in df[column_name].iteritems():
        value = time.gmtime(value)
    return df[column_name]

In [ ]:
def oversampling(df, column_name):
    oversampler = RandomOverSampler(sampling_strategy='auto', random_state=0)
    X = df.drop(column_name, axis=1)
    y = df[column_name]
    X_resampled, y_resampled = oversampler.fit_resample(X, y)
    resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name=column_name)], axis=1)
    return resampled_df

In [ ]:
def calculateAgeWithDateOfBirth(df,column_name):
    from datetime import datetime
    df[column_name]= pd.to_datetime(data[column_name])
    current_date= datetime.now()
    
    df['age_in_years']= (current_date-data[column_name]).astype('<m8[Y]')
    df['age_in_years'] = df['age_in_years'].map(lambda x: int(x))
    return df

In [ ]:
def makeGenderBinary(df, gender_column):
    df[gender_column]=df[gender_column].map(lambda x: 1 if x == 'M' else 0)
    return df

def convert_to_unixtime(df, datetime_column):

    df['unix_time'] = pd.to_datetime(df[datetime_column]).astype(int) // 10**9
    return df

In [ ]:
def splitDateAndTime(data,date_and_time_col):
    data[date_and_time_col] = pd.to_datetime(data[date_and_time_col])
    data['trans_date'] = data[date_and_time_col].dt.strftime('%Y-%m-%d')
    data['trans_date'] = pd.to_datetime(data['trans_date'])
    data['trans_time'] =data[date_and_time_col].dt.time
    return data

In [ ]:
def reduceDataWithPCA(data, components):
    from sklearn.decomposition import PCA
    reduced_data = pd.DataFrame(PCA(n_components=components).fit_transform(data))
    reduced_data.columns = ["PC"+str(i+1) for i in range(components)]
    return reduced_data

In [ ]:
data.columns

In [ ]:

## Cleaning the data and feature engineering

data = calculateAgeWithDateOfBirth(data,'dob')
data = makeGenderBinary(data, 'gender')

## if you want to over sample then don't uncomment these lines, the oversampler can not handle so many col
#data = encode_onehot(data,'job')
#data = encode_onehot(data,'category')
#data = splitDateAndTime(data,'trans_date_trans_time')
data = data.drop(columns=['Unnamed: 0','first','last','dob','city','street','state','trans_date_trans_time','trans_num','merchant','job','cc_num'])


## Splitting the data back to train and test after feature engineering and cleanup
train_cleaned = data[data['is_train']==1].drop(columns=['is_train'])
test_cleaned = data[data['is_train']==0].drop(columns=['is_train'])

print(len(train_cleaned), len(test_cleaned))

In [ ]:
#add data.to_csv here

In [ ]:
## RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

## Oversampling the data
oversampled_train = oversampling(train_cleaned,'is_fraud')
oversampled_train['is_fraud'].value_counts()
oversampled_train = encode_onehot(oversampled_train,'category')

oversampled_train_labels = oversampled_train['is_fraud']
oversampled_train_data = oversampled_train.drop(columns=['is_fraud'])

X_train, X_test, y_train, y_test = train_test_split(oversampled_train_data,oversampled_train_labels,test_size=0.3,train_size=0.7,random_state=5)

rf_1 = RandomForestClassifier(n_estimators=3, max_depth=5, random_state=0).fit(X_train, y_train)


In [ ]:
### ROC curve of rf_1

##One hot encoding the test dataset after feature engineering and cleanup
test_cleaned_onehot = encode_onehot(test_cleaned,'category')

## X and y validation dataframes for predicting probability
y_val = test_cleaned_onehot['is_fraud'].squeeze()
X_val = test_cleaned_onehot.drop(columns=['is_fraud'])# features
probs = rf_1.predict_proba(X_val)

probs = pd.DataFrame(probs)# Store in pandas Dataframe with two columns
probs = probs[1].squeeze() 

##finding unique cutoffs
cutoffs = pd.DataFrame({'cutoff':probs.unique()})
cutoffs = cutoffs.sort_values(by='cutoff',ascending=True)
##finding true postive rate and false positive rate
tpr = cutoffs.apply(lambda cut: np.sum(np.logical_and(probs >= cut[0], y_val == 1)) / np.sum(y_val == 1), axis=1)
fpr = cutoffs.apply(lambda cut: np.sum(np.logical_and(probs >= cut[0], y_val == 0)) / np.sum(y_val == 0), axis=1)

stats = pd.DataFrame({'cutoff':cutoffs.cutoff, 'tpr':tpr, 'fpr':fpr})
plt.plot(stats['fpr'],stats['tpr'])
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC curve for RandomForestClassifier with n_estimators=3 and max_depth=5')
plt.show()

print("Accuracy of rf_1 model on validation data from test_train_split: ",accuracy_score(rf_1.predict(X_test),y_test))
print("Accuracy of rf_1 model on test data from the separate file: ",accuracy_score(rf_1.predict(X_val),y_val))

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = y_val
y_pred = rf_1.predict(X_val)
print(test_data['is_fraud'].value_counts())
print(confusion_matrix(y_true,y_pred))

In [ ]:
from sklearn import tree

decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
tree.plot_tree(decision_tree)